In [ ]:
#Library Imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime
import os

#Load CSV file
def load_csv(path):
    if not os.path.exists(path):
        return pd.read_csv(path)

data_path = 'data/raw/marketingcampaigns.csv'
data = load_csv(data_path)
data.head()


